In [19]:
import pandas as pd
import numpy as np
import math
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, text, insert, Float, update, select
from bokeh.plotting import figure, show
from scipy.stats import linregress
from collections import Counter

file_path= "/Users/ajaychandraas/Desktop/Datasets written assignment-231-4 2/"

def engine_connect():
    engine= create_engine('sqlite:///main_code.db', echo=True)
    conn = engine.connect()
    return engine, conn
    
def create_train_table():
    path= file_path + "train.csv"
    df = pd.read_csv(path)
    create_table= df.to_sql("training_functions",engine_connect()[0])

def create_ideal_table():
    path= file_path + "ideal.csv"
    df= pd.read_csv(path)
    create_table= df.to_sql("ideal_functions",engine_connect()[0])

    
def create_test_table():
    """
    reading the test functions data from csv file, sorting the 'x' column values in ascending order
    removing the 'x' duplicate values by taking the mean 'y' values
    """
    path= file_path + "test.csv"
    df= pd.read_csv(path).sort_values(by='x').groupby('x', as_index=False).mean()
    """
    creating the schema for the test table
    """
    meta= MetaData()
    testTable= Table(
                'test_functions',meta,
                Column('index',Integer,primary_key=True),
                Column('x',Float),
                Column('y',Float),
                Column('Delta Y',Float),
                Column('No. of ideal func',String)
    )
    meta.create_all(engine_connect()[0])
    """
    adding the x,y rows line by line to the test table
    """
    for rec in range(0,len(df)):
        c1= df.iloc[rec][0]
        c2= df.iloc[rec][1]
        ins= testTable.insert().values(x=c1, y=c2)
        conn= engine_connect()[1]
        result= conn.execute(ins)

    
def get_train_table():
    conn= engine_connect()[1]
    get_train= pd.read_sql_table("training_functions", conn)
    return get_train

def get_ideal_table():
    conn= engine_connect()[1]
    get_ideal= pd.read_sql_table("ideal_functions", conn)
    return get_ideal

def get_test_table():
    conn= engine_connect()[1]
    get_test= pd.read_sql_table("test_functions", conn)
    return get_test

def drop_table(table_name):
    drp_table= text(f"DROP TABLE {table_name}")
    engine_connect()[0].execute(drp_table)


def train_ideal_mapping(): 
    final_dict={}
    train_col_name= get_train_table().columns.tolist()
    for iter_train in train_col_name:
        if(iter_train!= 'x' and iter_train!= 'index'):
            """fetching train columns one by one"""
            train_column= get_train_table().loc[:,iter_train].values
            train_dict={}
            ideal_col_name= get_ideal_table().columns.tolist()
            for iter_ideal in ideal_col_name:
                if(iter_ideal!= 'x' and iter_ideal!= 'index'):
                    """fetching ideal columns one by one"""
                    ideal_column= get_ideal_table().loc[:,iter_ideal].values
                    """
                    taking the difference between each column in train with each column in ideal 
                    and summing up using least squares to obtain the ideal functions
                    """
                    diff= sum((train_column-ideal_column)**2)
                    train_dict[iter_ideal]=diff
                    """
                    returning a dictionary with the training functions along with their respective ideal functions
                    """
                    min_value= min(train_dict.values())
                    min_keys = [key for key in train_dict if train_dict[key]==min_value]
                    final_dict[iter_train]=min_keys[0]
    return final_dict

    """
    Sum of least squares function output
    {'y1': 'y9', 'y2': 'y38', 'y3': 'y43', 'y4': 'y46'}

    """

def train_ideal_maxDeviation():
    train_dict={}    
    train_col_name= get_train_table().columns.tolist()
    for iter_train in train_col_name:
        if(iter_train!= 'x' and iter_train!= 'index'):
            """fetching train columns one by one"""
            ideal_fns= train_ideal_mapping()
            for iter_ideal in ideal_fns: 
                """
                matching the obtained 4 ideal functions with train functions and fetching the maximum value 
                in the difference between the absolute value of the train and ideal functions and multiplying 
                it with sqrt(2)
                """
                if(iter_ideal==iter_train):
                    train_column= get_train_table().loc[:,iter_train].values
                    ideal_column= get_ideal_table().loc[:,ideal_fns.get(iter_ideal)].values
                    diff= max(abs(train_column-ideal_column)) * math.sqrt(2) 
                    train_dict[ideal_fns.get(iter_ideal)]=diff
                    """
                    returning a dictionary with the ideal functions and the maximum deviation values with respect to its
                    training function
                    """
    return train_dict
            
    """ 
    train_ideal_maxDeviation() function output
    {'y9': 0.6992021759010193,
     'y38': 0.7065322852111048,
     'y43': 0.706170713229613,
     'y46': 0.7063877950114368}
     """

def test_ideal_abs_diff():
    """
    fetching the names of the ideal functions and the corresponding columns for ideal.csv file
    """
    mainList= []
    testTable_x_values= get_test_table().loc[:,'x'].values.tolist()

    mainList.append(testTable_x_values)

    for value in train_ideal_mapping().values():
        """
        get x and y values from sumOfLeastSquares function
        """
        ideal_x_values= get_ideal_table().loc[:,'x'].values;ideal_x_df= pd.DataFrame(ideal_x_values);
        ideal_column= get_ideal_table().loc[:,value].values;ideal_y_df= pd.DataFrame(ideal_column)
        # Finding regression line of ideal functions
        slope, intercept, r_value, p_value, std_err= linregress(ideal_x_values, ideal_column)
        x_arr_values= np.array(ideal_x_values)
        y_arr_values= np.array(ideal_column)
        est_y= (slope * x_arr_values) + intercept
        est_y_df= pd.DataFrame(est_y)
        y_dataframe= pd.concat([ideal_x_df, est_y_df],axis=1); y_dataframe.columns=['x','y']

        df1= y_dataframe; df2= get_test_table()
        merged_df= pd.merge(df1, df2, on='x');merged_df.columns=['x','y_ideal','index','y_test','dummy1','dummy2']
        diff= abs(merged_df['y_test']- merged_df['y_ideal']); final_diff= pd.DataFrame(diff);final_diff.columns=['y']
        final_df= pd.concat([merged_df['x'], final_diff],axis=1); final_df.columns=['x','y_diff1']
        y_values= final_diff.loc[:,'y'].values.tolist()
        mainList.append(y_values)
    final_table= pd.DataFrame(mainList).transpose();
        # Updating the column names for the tables
    ideal_value_head= train_ideal_mapping().values()
    ideal_value_list= []
    ideal_value_list.insert(0, 'x')
    for ideal_rec in ideal_value_head:
        ideal_value_list.append(ideal_rec)
    final_table.columns= ideal_value_list
    return final_table
    """
    test_ideal_abs_diff() function output
            x 	y9 	y38 	y43 	y46
    0 	-19.8 	3.464149 	3.557666 	1.619009 	0.683576
    1 	-19.6 	0.755766 	0.849745 	1.089348 	3.391934
    2 	-18.3 	28.140282 	28.043302 	29.985232 	32.287817
    3 	-18.0 	0.617949 	0.715621 	1.226964 	3.529549
    4 	-17.4 	4.847484 	4.946542 	3.002648 	0.700063
    ... 	... 	... 	... 	... 	...
    93 	17.8 	3.046066 	3.226383 	1.205677 	1.096909
    94 	17.9 	3.919520 	3.738972 	5.759896 	8.062481
    95 	18.1 	1.420794 	1.601803 	0.419558 	2.722143
    96 	18.4 	18.440353 	18.258651 	20.280666 	22.583251
    97 	18.8 	1.313830 	1.131205 	3.154093 	5.456678
    """  

def map_ideal_to_test():
    x_values= get_test_table().loc[:,'x'].values.tolist()
    y_values= get_test_table().loc[:,'y'].values.tolist()
    data_list= []
    meta = MetaData(bind=engine_connect()[0])
    MetaData.reflect(meta)
    conn= engine_connect()[1]
    
    ideal_list= []
    ideal_fn_names= train_ideal_mapping().values()
    for id_rec in ideal_fn_names:
        ideal_list.append(id_rec)
        
    for ideal_fn_rec in ideal_list:
        # Fetching max deviation values
        y_max= train_ideal_maxDeviation().get(ideal_fn_rec)
        # fetching the x and y values
        y_diff= test_ideal_abs_diff().loc[:,ideal_fn_rec].values.tolist()

        for x_rec, y_rec, y_val_rec in zip(x_values, y_diff, y_values):
            bulk_dict= {}
            """
            Checking if the value meets the max deviation criteria
            """   
            if(y_rec <= y_max):
                """
                Checking if x matches with more than 1 ideal function using select query
                """
                test_table= meta.tables['test_functions']
                sel= sqlalchemy.select(test_table).where(test_table.c.x == x_rec)
                check_x= conn.execute(sel).fetchall()[0][3]
                if(check_x == None):
                    """
                    Updating 'Delta Y' and 'Ideal func number' values
                    """
                    u = update(test_table)
                    u = u.values({ "Delta Y": y_rec, "No. of ideal func": ideal_fn_rec})
                    u= u.where(test_table.c.x == x_rec)
                    conn= engine_connect()[1]
                    result= conn.execute(u)
                    
                if(check_x != None ):
                    check_ideal_y= conn.execute(sel).fetchall()[0][3]
                    if(check_ideal_y != y_rec):
                        # Adding items to dictionary and then to the list to create data object
                        bulk_dict["x"]=x_rec
                        bulk_dict["y"]=y_val_rec
                        bulk_dict["Delta Y"]=y_rec
                        bulk_dict["No. of ideal func"]=ideal_fn_rec
                        data_list.append(bulk_dict)
                        
            
    ins= test_table.insert().values(data_list)
    conn= engine_connect()[1]
    res= conn.execute(ins)
    """
    Geting the unsorted test table, sorting the data based on 'x' column in aesc and updating the values
    back to the table
    """
    ord_list= []
    test_table= meta.tables['test_functions']
    sel= sqlalchemy.select(test_table).order_by(test_table.c.x.asc())
    fetch= conn.execute(sel).fetchall()
    for rec in fetch:
        ord_list.append(rec)
    # using a counter for matching the where clause
    count= 1
    for ord_rec in ord_list:
        if(ord_rec[4] != None):      
            u = update(test_table)
            u = u.values({ "x":ord_rec[1], "y":ord_rec[2], 
                          "Delta Y": ord_rec[3], "No. of ideal func": ord_rec[4]}).where(test_table.c.index == count)
            conn= engine_connect()[1]
            result= conn.execute(u)
            count+= 1
        else:
            u = update(test_table)
            u = u.values({ "x":ord_rec[1], "y":ord_rec[2], 
                          "Delta Y": 0, "No. of ideal func": "N/A"}).where(test_table.c.index == count)
            conn= engine_connect()[1]
            result= conn.execute(u)
            count+= 1
        
def mapping_test_points():
    meta = MetaData(bind=engine_connect()[0])
    MetaData.reflect(meta)
    conn= engine_connect()[1]
    test_table= meta.tables['test_functions']

    ideal_list= []
    ideal_fn_names= train_ideal_mapping().values()
    for id_rec in ideal_fn_names:
        ideal_list.append(id_rec)

    for ideal_rec in ideal_list:
        # Plotting test values into a scatter plot
        sel= sqlalchemy.select(test_table).where(test_table.c['No. of ideal func'] == ideal_rec)
        sel_y= conn.execute(sel).fetchall()
        xlst= []; ylst= []
        for rec in sel_y:
            xlst.append(rec[1])
            ylst.append(rec[2])
        
        max_dev_val= round(train_ideal_maxDeviation()[ideal_rec],3)
        title= f"{ideal_rec} Scatter plot with ideal best fit line with max boundary of {max_dev_val}"
        p= figure(title= title, x_axis_label='X axis', y_axis_label='Y axis')
        p.title.text_font_size = '13pt'
        p.scatter(xlst, ylst, size= 10)

        # Adding ideal y values regression line to the scatter plot
        ideal_list= [ideal_rec]
        for plot_rec in ideal_list: 
            ideal_x= get_ideal_table().loc[:,'x'].values
            ideal_y= get_ideal_table().loc[:,plot_rec].values
            slope, intercept, r_value, p_value, std_err= linregress(ideal_x, ideal_y)
            xs = np.linspace(np.min(ideal_x), np.max(ideal_x), 100)
            ys = slope * xs + intercept
            p.line(xs, ys, color='red')

            # Adding maximum deviation as upper and lower bounds to the scatter plot
            x_values= np.array(ideal_x)
            est_y= (slope * x_values) + intercept 
            pos_dev_est= est_y + train_ideal_maxDeviation()[plot_rec]
            neg_dev_est= est_y - train_ideal_maxDeviation()[plot_rec]
            p.line(ideal_x, pos_dev_est, color='red')
            p.line(ideal_x, neg_dev_est, color='red')

        show(p)    
            
def test_ideal_mapping_count():
    mapped_ideal_val= get_test_table().loc[:,'No. of ideal func'].values
    result_dict = dict(Counter(mapped_ideal_val))

    mapped_ideal_x_val= get_test_table().loc[:,'x'].values
    x_counter= dict(Counter(mapped_ideal_x_val))
    rep_count= 0
    for rec in x_counter.values():
        if(rec > 1):
            rep_count+= 1

    result_dict['multiple mappings']= rep_count
    print(result_dict)             
    """
    Output
    {'y46': 9, 'N/A': 48, 'y9': 12, 'y38': 9, 'y43': 20, 'multiple mappings': 8}

    """       

# engine_connect()
# create_train_table()
# create_ideal_table()
# create_test_table()
# train_ideal_mapping()
# train_ideal_maxDeviation()
# test_ideal_abs_diff()
# map_ideal_to_test()
# mapping_test_points()

# test_ideal_mapping_count() 


2023-03-28 19:04:13,846 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("training_functions")
2023-03-28 19:04:13,847 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:13,848 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("training_functions")
2023-03-28 19:04:13,848 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:13,849 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-28 19:04:13,849 INFO sqlalchemy.engine.Engine [raw sql] ('training_functions',)
2023-03-28 19:04:13,850 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("training_functions")
2023-03-28 19:04:13,850 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:13,850 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("training_functions")
2023-03-28 19:04:13,851 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:13,851 INFO sqlalchemy.engine.Engine S

2023-03-28 19:04:13,900 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:13,901 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-28 19:04:13,901 INFO sqlalchemy.engine.Engine [raw sql] ('ideal_functions',)
2023-03-28 19:04:13,902 INFO sqlalchemy.engine.Engine PRAGMA main.index_list("ideal_functions")
2023-03-28 19:04:13,902 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:13,903 INFO sqlalchemy.engine.Engine PRAGMA main.index_info("ix_ideal_functions_index")
2023-03-28 19:04:13,903 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:13,905 INFO sqlalchemy.engine.Engine PRAGMA main.index_list("ideal_functions")
2023-03-28 19:04:13,906 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:13,907 INFO sqlalchemy.engine.Engine PRAGMA main.index_info("ix_ideal_functions_index")
2023-03-28 19:04:13,908 INFO sqlalchemy.engine.Engine [raw sql] (

2023-03-28 19:04:13,988 INFO sqlalchemy.engine.Engine [generated in 0.00035s] ()
2023-03-28 19:04:14,007 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ideal_functions")
2023-03-28 19:04:14,007 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:14,009 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("ideal_functions")
2023-03-28 19:04:14,009 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:14,011 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-28 19:04:14,012 INFO sqlalchemy.engine.Engine [raw sql] ('ideal_functions',)
2023-03-28 19:04:14,012 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("ideal_functions")
2023-03-28 19:04:14,013 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:14,013 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("ideal_functions")
2023-03-28 19:04:14,013 INFO sqlalchemy.engine.Engine [r

2023-03-28 19:04:14,081 INFO sqlalchemy.engine.Engine PRAGMA main.index_info("ix_ideal_functions_index")
2023-03-28 19:04:14,081 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:14,082 INFO sqlalchemy.engine.Engine PRAGMA main.index_list("ideal_functions")
2023-03-28 19:04:14,082 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:14,083 INFO sqlalchemy.engine.Engine PRAGMA main.index_info("ix_ideal_functions_index")
2023-03-28 19:04:14,083 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:14,084 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-28 19:04:14,084 INFO sqlalchemy.engine.Engine [raw sql] ('ideal_functions',)
2023-03-28 19:04:14,086 INFO sqlalchemy.engine.Engine SELECT ideal_functions."index", ideal_functions.x, ideal_functions.y1, ideal_functions.y2, ideal_functions.y3, ideal_functions.y4, ideal_functions.y5, ideal_functions.y6, ide

2023-03-28 19:04:14,159 INFO sqlalchemy.engine.Engine [generated in 0.00055s] ()
2023-03-28 19:04:14,175 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ideal_functions")
2023-03-28 19:04:14,175 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:14,177 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("ideal_functions")
2023-03-28 19:04:14,177 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:14,180 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-28 19:04:14,180 INFO sqlalchemy.engine.Engine [raw sql] ('ideal_functions',)
2023-03-28 19:04:14,181 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("ideal_functions")
2023-03-28 19:04:14,181 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:14,181 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("ideal_functions")
2023-03-28 19:04:14,182 INFO sqlalchemy.engine.Engine [r

...

2023-03-28 19:04:41,246 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ()
2023-03-28 19:04:41,264 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ideal_functions")
2023-03-28 19:04:41,265 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:41,266 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("ideal_functions")
2023-03-28 19:04:41,266 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:41,268 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-28 19:04:41,268 INFO sqlalchemy.engine.Engine [raw sql] ('ideal_functions',)
2023-03-28 19:04:41,269 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("ideal_functions")
2023-03-28 19:04:41,269 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:41,270 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("ideal_functions")
2023-03-28 19:04:41,270 INFO sqlalchemy.engine.Engine [r

2023-03-28 19:04:41,335 INFO sqlalchemy.engine.Engine PRAGMA main.index_info("ix_ideal_functions_index")
2023-03-28 19:04:41,336 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:41,336 INFO sqlalchemy.engine.Engine PRAGMA main.index_list("ideal_functions")
2023-03-28 19:04:41,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:41,337 INFO sqlalchemy.engine.Engine PRAGMA main.index_info("ix_ideal_functions_index")
2023-03-28 19:04:41,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:41,338 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-28 19:04:41,338 INFO sqlalchemy.engine.Engine [raw sql] ('ideal_functions',)
2023-03-28 19:04:41,340 INFO sqlalchemy.engine.Engine SELECT ideal_functions."index", ideal_functions.x, ideal_functions.y1, ideal_functions.y2, ideal_functions.y3, ideal_functions.y4, ideal_functions.y5, ideal_functions.y6, ide

2023-03-28 19:04:41,406 INFO sqlalchemy.engine.Engine [generated in 0.00037s] ()
2023-03-28 19:04:41,425 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ideal_functions")
2023-03-28 19:04:41,425 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:41,426 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("ideal_functions")
2023-03-28 19:04:41,426 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:41,429 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-28 19:04:41,429 INFO sqlalchemy.engine.Engine [raw sql] ('ideal_functions',)
2023-03-28 19:04:41,430 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("ideal_functions")
2023-03-28 19:04:41,430 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-28 19:04:41,431 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("ideal_functions")
2023-03-28 19:04:41,431 INFO sqlalchemy.engine.Engine [r

2023-03-28 19:04:41,489 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ()


{'y9': 0.6992021759010193,
 'y38': 0.7065322852111048,
 'y43': 0.706170713229613,
 'y46': 0.7063877950114368}